In [13]:
import tensorflow as tf
from tensorflow.keras.utils import get_file
from tensorflow.keras.preprocessing import image_dataset_from_directory

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import Rescaling
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import matplotlib.pyplot as plt

In [2]:
# 데이터 불러오기
train_dir = "data/train/train"
val_dir = "data/validation/validation"

### 데이터셋 shape 분석
- 32 : 이미지의 개수 (한 배치 안에 32장)
- 224 : height (픽셀)
- 224 : width (픽셀)
- 3 : 채널 수(RGB)

- (32,) : 각 이미지가 어떤 클래스에 속하는지 알려주는 정답 정보(=label)

In [3]:
# 학습 데이터 생성
train_dataset = image_dataset_from_directory(
    train_dir,              # 학습 데이터 디렉토리 경로
    image_size=(224, 224),  # 이미지 크기
    batch_size=32,          # 배치 사이즈(한번에 처리할 이미지 크기)
    shuffle=True,           # 데이터 섞기
    seed=0                  # 재현성을 위한 시드 설정
)

# 검증 데이터 생성
val_dataset = image_dataset_from_directory(
    val_dir,                # 검증 데이터 디렉토리 경로
    image_size=(224, 224),  # 이미지 크기
    batch_size=32,          # 배치 사이즈(한번에 처리할 이미지 크기)
    shuffle=True,           # 데이터 섞기
    seed=0                  # 재현성을 위한 시드 설정
)

Found 1034 files belonging to 3 classes.
Found 133 files belonging to 3 classes.


In [4]:
class_names = train_dataset.class_names
print(class_names)

['angular_leaf_spot', 'bean_rust', 'healthy']


In [5]:
# 모델 layer 구성
model = Sequential([
    Rescaling(1./255, input_shape=(224, 224, 3)),   # 이미지 정규화
    Conv2D(32, (3, 3), activation='relu'),          # 첫 번째 합성곱 층
    MaxPooling2D((2, 2)),                           # 풀링 층
    Conv2D(64, (3, 3), activation='relu'),          # 두 번째 합성곱 층
    MaxPooling2D((2, 2)),                           # 풀링 층
    Conv2D(128, (3, 3), activation='relu'),         # 세 번째 합성곱 층
    MaxPooling2D((2, 2)),                           # 풀링 층
    Flatten(),                                       # 평탄화 층
    Dense(128, activation='relu'),                  # 완전 연결층
    Dropout(0.5),                                   # 드롭아웃 층
    Dense(len(class_names), activation='softmax')   # 출력층
])

In [6]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       7

In [7]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [14]:
early_stopping = EarlyStopping(patience=3)
model_checkpoint = ModelCheckpoint("ModelCheckpoint.keras")

### 주요 모델 저장 포맷 종류
1) HDF5 포맷: 'model.h5'
    - 가중치 + 구조 저장 가능
2) SavedModel 포멧: 'model.keras'
    - 2022: 가장 최신 모델 포맷

In [16]:
# 모델 훈련
hist = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=10,
        callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/10


33/33 [==============================] - 18s 519ms/step - loss: 1.4279 - accuracy: 0.4130 - val_loss: 0.8678 - val_accuracy: 0.6466
Epoch 2/10
33/33 [==============================] - 15s 447ms/step - loss: 0.8521 - accuracy: 0.6238 - val_loss: 0.6913 - val_accuracy: 0.7368
Epoch 3/10
33/33 [==============================] - 15s 446ms/step - loss: 0.7579 - accuracy: 0.6663 - val_loss: 0.6571 - val_accuracy: 0.7293
Epoch 4/10
33/33 [==============================] - 15s 452ms/step - loss: 0.6962 - accuracy: 0.7108 - val_loss: 0.7429 - val_accuracy: 0.6466
Epoch 5/10
33/33 [==============================] - 15s 440ms/step - loss: 0.6268 - accuracy: 0.7408 - val_loss: 0.6075 - val_accuracy: 0.7218
Epoch 6/10
33/33 [==============================] - 16s 466ms/step - loss: 0.5749 - accuracy: 0.7795 - val_loss: 0.5578 - val_accuracy: 0.7820
Epoch 7/10
33/33 [==============================] - 16s 482ms/step - loss: 0.5171 - accuracy: 0.7872 - val_loss: 0.5507 - val_accuracy: 0.7744
Epoch 8/10

### fit의 핵심 3요소

#### 1. epochs: 총 복습 횟수
**epoch**는 전체 훈련 데이터셋을 몇 번 반복해서 학습할지를 결정함
- 역할: 모델이 데이터를 충분히 학습할 기회를 제공한다.
- 비유: 학생이 시험을 보기 위해 교과서를 몇 번 정독할지 정하는 것
- 중요성:
    - 너무 적으면(Underfitting): 모델이 데이터의 패턴을 충분히 학습하지 못함(과소적합)
    - 너무 크면(Overfitting): 모델이 훈련 데이터에만 과도하게 최적화되어 새로운 데이터에 대한 성능이 떨어짐(과적합)

#### 2. EarlyStopping: 학습 중단 신호
**earlystopping**은 모델의 성능이 더 이상 개선되지 않으면 학습을 조기에 중단시킴
- 역할: 과적합 방지, 불필요한 학습 시간 절약
- 비유: 학생이 매주 모의고사를 공부했는데 점수가 오르지 않고 떨어지면 공부를 멈추는게 효율적이라고 판단하는 것
- 주요 설정:
    - monitor='val_loss': 이 값이 더 이상 감소하지 않으면 멈출 준비를 함
    - patience=10: 점수가 오르지 않아도 10번의 epoch를 참고 기다려 줌
    - restore_bset_weights=True **(매우 중요)**: 학습이 중단되었을 때, 지금까지 중 가장 좋았던 때의 가중치로 모델을 되돌려줌

#### 3. ModelCheckpoint: 최고의 순간을 저장
**ModelCheckpoint**는 학습 과정 동안 모델의 상태를 파일로 저장함
- 역할: 학습 중 최고의 성능을 보인 모델을 저장하거나, 예기치 않게 학습이 중단될 경우를 대비해 중간 과정 저장
- 비유: 학생이 공부하다가 컨디션이 좋고 문제가 잘 풀렸던 "최고의 순간"을 기록해두는 것
- 주요 설정:
    - filepath='best_model.keras': 모델을 저장할 파일 경로와 이름
    - monitor='val_loss': 어떤 지표로 '최고'를 정할 건지 지정
    - save_best_only=True: 성능이 더 좋아지면 그 모델을 덮어씌움
    - save_weights_only=False: 모델의 구조, 가중치 등 전체 모델 저장 (True=가중치만 저장)

In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib as mp

print("TensorFlow version:", tf.__version__)
print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("Matplotlib version:", mp.__version__)

TensorFlow version: 2.10.0
NumPy version: 1.24.3
Pandas version: 2.0.3
Matplotlib version: 3.7.0
